In [1]:
%load_ext blackcellmagic

In [3]:
from flask import Flask, request, render_template, jsonify, json
from decouple import config
from flask_cors import CORS
import numpy as np
import networkx as nx
import tweepy
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from collections import Counter
import asyncio
from concurrent.futures import ThreadPoolExecutor
import requests
import nest_asyncio
nest_asyncio.apply()

#set up twitter environment
TWITTER_AUTH =tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))
TWITTER = tweepy.API(TWITTER_AUTH)

In [4]:
def get_user_timeline(session, search, edge_list, page):
    timeline = TWITTER.user_timeline(
        id=search,
        count=200,
        exclude_replies=False,
        include_rts=True,
        tweet_mode="extended",
        page=page,
    )
    timeline=[
            user_mentions["screen_name"]
            for um_list in [tweet.entities["user_mentions"] for tweet in timeline]
            for user_mentions in um_list
        ]
    #timeline = [i for sublist in timeline for i in sublist]
    edge_list += [(search,i) for i in timeline]
    global top_interactions
    top_interactions = [i for i in Counter(timeline).most_common(10) ]
    top_interactions = [ i[0] for i in top_interactions]
    return top_interactions, edge_list

def get_user_favorites(session, search, edge_list, page):
    timeline = TWITTER.favorites(
        id=search,
        count=200,
        exclude_replies=False,
        include_rts=True,
        tweet_mode="extended",
        page=page,
    )
    timeline = [tweet.author.screen_name for tweet in timeline]
    edge_list += [ (i,search) for i in timeline]
    return edge_list

In [5]:
async def get_input_user_data_asynchronous(search, edge_list):
    with ThreadPoolExecutor(max_workers=20) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_user_timeline, *(session, search, edge_list, page)
                )
                for page in range(10)
            ] + [
                loop.run_in_executor(
                    executor, get_user_favorites, *(session, search,edge_list, page)
                )
                for page in range(10)
            ]
            for response in await asyncio.gather(*tasks):
                pass

def execute_async_input_user_event_loop(search, edge_list):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.ensure_future(get_input_user_data_asynchronous(search, edge_list))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    return edge_list, top_interactions

In [10]:
async def get_interactions_data_asynchronous(top_interactions, edge_list):
    with ThreadPoolExecutor(max_workers=20) as executor:
        with requests.Session() as session:
            loop = asyncio.get_event_loop()
            tasks = [
                loop.run_in_executor(
                    executor, get_user_timeline, *(session, search, edge_list, 1)
                )
                for search in top_interactions
            ] + [
                loop.run_in_executor(
                    executor, get_user_favorites, *(session, search, edge_list, 1)
                )
               for search in top_interactions
            ]
            for response in await asyncio.gather(*tasks):
                pass
            
def execute_async_interactions_event_loop(top_interactions, edge_list):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.ensure_future(get_interactions_data_asynchronous(top_interactions, edge_list))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)
    #index = [i for sublist in index for i in sublist]
    #top_interactions = [ i[0] for i in Counter(index).most_common(5)
    return edge_list

In [11]:
def main(search):
    edge_list =[]
    edge_list, top_interactions = execute_async_input_user_event_loop(search,edge_list)
    edge_list = execute_async_interactions_event_loop(top_interactions,edge_list)
    return edge_list
    

In [13]:
%%time
edge_list= main("austen")

Wall time: 2.23 s


In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(edge_list)
pd.set_option("display.max_rows", len(df))

In [16]:
df

,0,1
0,austen,paulg
1,austen,robertnessler
2,austen,cyantist
3,austen,davidzmorris
4,austen,sharont33370098
5,austen,LambdaSchool
6,austen,patio11
7,austen,shaunmmaguire
8,austen,runvc
9,austen,CanadaKaz
